In [ ]:
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf
from tensorflow.keras import backend as K

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0' # Set to -1 if CPU should be used CPU = -1 , GPU = 0

gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
elif cpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        logical_cpus= tf.config.experimental.list_logical_devices('CPU')
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# Set the image size with are learning from
IMG_WIDTH, IMG_HEIGHT = 224, 224

# Set the constants
TRAIN_DATA_DIR = '../input/animals/train'
VALIDATION_DATA_DIR = '../input/animals/validation'

EPOCHS = 15
BATCH_SIZE = 100
NUM_CLASSES = 16

# Machine Learning Model Filename
MODEL_FILENAME = './saved_model.h5'

In [ ]:
def build_model():
    # to check if first dim is channel or not
    if K.image_data_format() == 'channels_first':
        input_shape = (3, IMG_WIDTH, IMG_HEIGHT)
    else:
        input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)
    
    # start to build architecture of model
    model = Sequential()
    
    model.add(tf.keras.applications.EfficientNetB2(
        input_shape=input_shape,
        include_top=False,
        weights="imagenet",
        classes=NUM_CLASSES,
        classifier_activation="softmax"
    ))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    # make training configuration
    model.compile(loss='binary_crossentropy', 
                  optimizer=tf.keras.optimizers.SGD(lr=0.01), 
                  metrics=['accuracy']
                 )
    return model

In [ ]:
def train_model(model):
    # data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        validation_split=0.1,
        rotation_range=30,
        shear_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    validation_generator = train_datagen.flow_from_directory(
        directory=TRAIN_DATA_DIR,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        subset='validation',
        shuffle=True,
    )
    
    # start training
    model.fit(
        train_generator,
        validation_data=train_generator,
        steps_per_epoch = train_generator.n//BATCH_SIZE,
        validation_steps = validation_generator.n//BATCH_SIZE,
        epochs=EPOCHS,
        workers=2
    )
    
    return model

In [ ]:
def evaluate(model):

    model = tf.keras.models.load_model('./saved_model.h5')

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    test_generator = test_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    )

    loss, accuracy = model.evaluate_generator(test_generator)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)

In [ ]:
def main():
    myModel = build_model()
    myModel = train_model(myModel)
    myModel.save(MODEL_FILENAME)
    evaluate(myModel)
    
main()